<a href="https://colab.research.google.com/github/Rish-23072005/ML-PROJECT-1/blob/main/Music_Generation_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#IMPORTING LIBRARIES

In [ ]:
import os
import zipfile
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
import music21 as m21

# Specify the path to the MIDI dataset zip file

In [ ]:
zip_file_path = '/content/Music Dataset.zip'
# Extract the MIDI dataset
extracted_folder_path = '/content/folder'
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)

# Load and preprocess the MIDI dataset

In [ ]:
def load_midi_dataset(dataset_folder):
    all_sequences = []

    for root, _, files in os.walk(dataset_folder):
        for file in files:
            if file.endswith('.mid') or file.endswith('.midi'):
                file_path = os.path.join(root, file)
                midi = m21.converter.parse(file_path)
                notes = []

                for element in midi.flat.notes:
                    if isinstance(element, m21.note.Note):
                        notes.append(str(element.pitch))
                    elif isinstance(element, m21.chord.Chord):
                        notes.append('.'.join(str(n) for n in element.pitches))

                if len(notes) > 0:
                    all_sequences.append(notes)

    return all_sequences

dataset_folder = extracted_folder_path  # path to the extracted dataset folder
all_sequences = load_midi_dataset(dataset_folder)

/usr/local/lib/python3.10/dist-packages/music21/midi/translate.py:874: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=4, channel=None, data=b'Copyright \xa9 2001 by Bernd Kr\xfcger'>; getting generic Instrument
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/music21/stream/base.py:3694: Music21DeprecationWarning: .flat is deprecated.  Call .flatten() instead
  return self.iter().getElementsByClass(classFilterList)
/usr/local/lib/python3.10/dist-packages/music21/midi/translate.py:874: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=5, channel=None, data=b'Copyright \xa9 2010 by Bernd Krueger'>; getting generic Instrument
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/music21/midi/translate.py:874: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=5, channel=None, data=b'Copyright \xa9 2009 by Bernd Kr

# Create a vocabulary of unique notes and chords

In [ ]:
notes_and_chords = set(n for seq in all_sequences for n in seq)
num_unique_notes = len(notes_and_chords)
note_to_index = {note: index for index, note in enumerate(sorted(notes_and_chords))}
index_to_note = {index: note for index, note in enumerate(sorted(notes_and_chords))}


# Convert the MIDI sequences to numerical input/output pairs

In [ ]:
input_sequences = []
output_sequences = []

sequence_length = 100  # Specify the length of the input sequences
for sequence in all_sequences:
    for i in range(len(sequence) - sequence_length):
        input_seq = sequence[i : i + sequence_length]
        output_seq = sequence[i + sequence_length]
        input_sequences.append([note_to_index[note] for note in input_seq])
        output_sequences.append(note_to_index[output_seq])


# Convert the input/output sequences to numpy arrays

In [ ]:
input_sequences = np.array(input_sequences)
output_sequences = np.array(output_sequences)


# Normalize the input sequences

In [ ]:
input_sequences = input_sequences / float(num_unique_notes)

# Split the dataset into training and validation sets

In [ ]:
split_index = int(len(input_sequences) * 0.8)
train_input = input_sequences[:split_index]
train_output = output_sequences[:split_index]
val_input = input_sequences[split_index:]
val_output = output_sequences[split_index:]

# Define the CNN model

In [ ]:
model = tf.keras.Sequential()
model.add(layers.Embedding(input_dim=num_unique_notes, output_dim=128, input_length=sequence_length))
model.add(layers.Conv1D(filters=128, kernel_size=5, strides=1, padding='causal', activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(layers.Conv1D(filters=256, kernel_size=5, strides=1, padding='causal', activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(layers.Conv1D(filters=512, kernel_size=5, strides=1, padding='causal', activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(num_unique_notes, activation='softmax'))

# Compile the model

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

# Train the model

In [ ]:
model.fit(train_input, train_output, epochs=20, batch_size=64, validation_data=(val_input, val_output))

Epoch 1/20
5653/5653 [==============================] - 67s 12ms/step - loss: 6.0423 - val_loss: 6.1725
Epoch 2/20
5653/5653 [==============================] - 65s 12ms/step - loss: 6.0419 - val_loss: 6.1816
Epoch 3/20
5653/5653 [==============================] - 66s 12ms/step - loss: 6.0412 - val_loss: 6.1836
Epoch 4/20
5653/5653 [==============================] - 65s 12ms/step - loss: 6.0400 - val_loss: 6.1838
Epoch 5/20
5653/5653 [==============================] - 66s 12ms/step - loss: 6.0394 - val_loss: 6.1812
Epoch 6/20
5653/5653 [==============================] - 66s 12ms/step - loss: 6.0387 - val_loss: 6.1892
Epoch 7/20
5653/5653 [==============================] - 65s 12ms/step - loss: 6.0380 - val_loss: 6.1880
Epoch 8/20
5653/5653 [==============================] - 64s 11ms/step - loss: 6.0373 - val_loss: 6.1901
Epoch 9/20
5653/5653 [==============================] - 65s 11ms/step - loss: 6.0364 - val_loss: 6.1909
Epoch 10/20
5653/5653 [==============================] - 65s 11m

# Generate new music

In [ ]:
def generate_music(model, seed_sequence, num_notes):
    generated_sequence = list(seed_sequence)
    model.reset_states()
    for i in range(num_notes):
        # Convert the sequence to a 3D array as required by the model
        input_seq = np.expand_dims(np.array(generated_sequence[-sequence_length:]), axis=0)
        # Predict the probability distribution over the next note
        predicted_probs = model.predict(input_seq)[0]
        # Sample the next note based on the predicted probabilities
        predicted_index = np.random.choice(range(num_unique_notes), p=predicted_probs)
        generated_sequence.append(predicted_index)
    return np.array(generated_sequence)

 # Generate a seed sequence to start the music generation
#Create a MIDI file from the generated notes

In [ ]:

seed_length = sequence_length
seed_sequence = np.random.randint(num_unique_notes, size=seed_length)

# Generate music
generated_sequence = generate_music(model, seed_sequence, num_notes=100)

# Convert the generated sequence back to notes
generated_notes = [index_to_note[index] for index in generated_sequence]

# Create a MIDI file from the generated notes
midi_stream = m21.stream.Stream()

for note in generated_notes:
    if '.' in note:
        chord_notes = note.split('.')
        chord = m21.chord.Chord(chord_notes)
        midi_stream.append(chord)
    else:
        new_note = m21.note.Note(note)
        midi_stream.append(new_note)

output_midi_path = '/content/folder.mid'  # Specify the output MIDI file path
midi_stream.write('midi', fp=output_midi_path)

print('Music generated and saved as MIDI file:', output_midi_path)

1/1 [==============================] - 0s 19ms/step
Music generated and saved as MIDI file: /content/folder.mid
